# Python concurrency overview

Due to limitations from physics it's been harder and harder to get more CPU performance by increasing clock speed and adding more power. As a result modern CPU design has very strongly moved in the direction of adding more CPU cores:

In [3]:
import os
cpu_count = os.cpu_count()
print(f"we have {cpu_count} CPUs, how do we use them all at once?")

we have 12 CPUs, how do we use them all at once?


On this laptop for example there's multiple CPU cores and even lower end devices, including many phone models, are starting to introduce multiple CPU cores. In the early years of Python in the early 1990's there were far fewer multi core systems and as hardware has evolved so have the concurrency options of Python.

If you have a situation where you need to run multiple computations at the same time, Python gives you the following concurrency options (including but not limited to):

- asyncio (Since python 3.5)
- Python threads
- GIL-released threads
- multiprocessing
- distributed tasks

## The "minimum schedulable unit"

The minimum schedulable unit of work is the smallest chunk of work that can be scheduled to be run.

More precisely it's individual semantic units of code that can be decided to be executed by the scheduler before control is returned to the scheduler.

This is an important concept because for concurrency we have to both define what code will run and also *when that code is run*.

## Asyncio

- Built in language support since Python 3.5 (partially since Python 3.4)
- There's only one event loop and thus only one co-routine runs at any given time
- MSU: "awaitable block"
- event loop runs these awaitable blocks

Good when:

- You problem is IO bound
- You are starting a new codebase without synchronous legacy code

Downsides:

- If code is inherently synchronous like accessing a database then asyncio can degrade performance compared to using threads

What this *doesn't* solve:

- Running code across multiple cores simultaneously, asyncio runs in one thread only.

In [2]:
import asyncio
async def print_after_sleep(to_print: str):
    await asyncio.sleep(5)
    print (to_print)

loop = asyncio.get_event_loop()

print_hello = loop.create_task(print_after_sleep("Hello"))
try:
    loop.run_until_complete(print_hello)
finally:
    print("Closing loop")
    loop.close()

Closing loop


RuntimeError: Cannot close a running event loop

Hello


## Python threads

Like many other languages Python supplies the ability to create threads. There's some issues however with the Global Interpreter Lock (commonly referred to by the acronym GIL) that substantially impacts the performance of threaded code in Python.

- One thread runs (Due to the GIL)
- Some things release the GIL however like sleeps and IO
- MSU: python bytecode
- global state is shared but is only consistent for single-bytecode operations
- combined scheduling

Good when:

- You need preemptive multitasking (be able to interrupt other threads)
- Integrating synchronous code
- Situations which require fine grained concurrency
- Python glue to other languages like C/Rust/ etc

Downsides:

- Can exhaust the virtual memory on 32 bit machines if you start a lot of threads since each thread is quite heavy on virtual memory.

What this *doesn't* solve:

- CPU bound concurrency if the GIL is not released

The GIL presents a big gotcha and performance can be substantially degraded by introducing threads if the GIL is not released. This is because the locking happens in the python interpreter itself so only one piece of bytecode can be evaluated at once. Because you can induce the thread scheduling and context switching
overhead for effectively no benefit in this case performance can greatly suffer. Consider multiprocessing if you need multiple cores to be utilized simultaneously.

## GIL released threads

- Multiple GIL released threads *can* run simultaneously (since there's no lock preventing this)
- MSU: host processor instruction
- global state is shared but is unreliable
- OS-scheduled

This is what allows libraries such as Numpy to be so fast, the Numpy library calls compiled code that does not have any GIL limitations.

This is mostly something that library implementers have to consider, for example FFI calls.

Python functions implemented in C, with `Py_BEGIN_ALLOW_THREADS`, will not acquire a the global interpreter lock.

Good when:

- Using 3rd party libraries like numpy that let you get the benefits of fast compiled code that will not cause the Python interpreter to lock

Downsides:

- The GIL exists to protect the internal state of the Python interpreter itself, so if you don't acquire these locks you have less protections when things go wrong. Specifically the failure modes can involve the Python interpreter itself crashing if you get things wrong when you write this sort of code.

## Multiprocessing

- multiple processes run simultaneously
- child processes are copied over completely, this however leads to situations where the global state isn't actually shared, it starts the same but can/will change over time

See:

In [ ]:
import os
os.fork

Also see `ProcessPoolExecutor` from the `concurrent.futures` library in the standard library

Good when:

- You have tasks that can be executed in parallel without needing substantial inter-task communication
- You have CPU bound tasks that benefit with the ability to have bytecode interpreted by the runtime run fully unhindered by the internal interpreter locks (GIL)

Downsides:

- Can run into issues with RAM usage due the fact that a complete copy of program state is created for each process by default. For example if your script takes 1GB of RAM then you create 8 processes you will now have `1GB*8 = 8GB` of RAM usage. If RAM usage becomes a bottleneck you may have to use something like mmap backed storage to share state across processes, this comes with its own issues however.

What this doesn't solve:

- Not a great solution for IO bound latency, you'll pay a substantial memory overhead for multiple processes but won't get much of a win over asyncio in a IO bound case.

## Distributed tasks

- eg Dask, Hadoop
- MSU varies, often the entire application for some subset of data
- central orchestrator

Good when:

- You have a highly segmentable and distributable workload
- The need for shared state is minimal (network latency is exceedingly bad in this case)
- Data sets are too big to handle in memory easily

Downsides:

- Running any computation over multiple machines introduces significant complexity. For example you can't rely on function calls succeeding due to issues like network timeouts. This can make error handling substantially more complex.
- Failure modes require different debugging techniques since they might not be in the same process or even same machine anymore.